In [2]:
# general imports
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import os
import pandas as pd

# dataset
from sklearn.datasets import fetch_kddcup99
from sklearn.preprocessing import StandardScaler

# pyspark module
from pyspark.rdd import RDD

# src module
from src.utils import sparkSetup
from src.kmeans import compute_centroidDistances, get_clusterId, get_minDistance

In [3]:
# creating the zipped environment if it doesn't already exist
! if [ ! -f "environment.tar.gz" ]; then venv-pack -o "environment.tar.gz" ; fi
# creating the zipped module src
! if [ -f "src.tar.gz" ]; then rm src.tar.gz ; fi
! tar -czf src.tar.gz src

In [4]:
# starting the cluster
! $SPARK_HOME/sbin/start-all.sh

starting org.apache.spark.deploy.master.Master, logging to /usr/local/spark/logs/spark-ubuntu-org.apache.spark.deploy.master.Master-1-mapd-b-14-1.out


master: starting org.apache.spark.deploy.worker.Worker, logging to /usr/local/spark/logs/spark-ubuntu-org.apache.spark.deploy.worker.Worker-1-mapd-b-14-1.out
worker3: starting org.apache.spark.deploy.worker.Worker, logging to /usr/local/spark/logs/spark-ubuntu-org.apache.spark.deploy.worker.Worker-1-mapd-b-14-4.out
worker2: starting org.apache.spark.deploy.worker.Worker, logging to /usr/local/spark/logs/spark-ubuntu-org.apache.spark.deploy.worker.Worker-1-mapd-b-14-3.out
worker1: starting org.apache.spark.deploy.worker.Worker, logging to /usr/local/spark/logs/spark-ubuntu-org.apache.spark.deploy.worker.Worker-1-mapd-b-14-2.out


In [5]:
# telling spark where to find the python binary
os.environ["PYSPARK_PYTHON"] = "./environment/bin/python"

In [6]:
# creating a sparkSession
spark = sparkSetup("kMeans")
sc = spark.sparkContext
# exporting the src module
sc.addPyFile("src.tar.gz") 

25/09/09 19:37:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/09 19:37:35 WARN Utils: Untarring behavior will be deprecated at spark.files and SparkContext.addFile. Consider using spark.archives or SparkContext.addArchive instead.


### Naming conventions

The single datum is named as `datumName`, while the RDD that is a collection of one or more data is called `datumName_rdd`.

Example: `compute_clusterDistances` returns `clusterDistances` (i.e. a numpy array of distances between a point `x` and the `centroids`). 
The RDD that collects all the `clusterDistances` will be called `clusterDistances_rdd`, and here is a sample implementation of that:
```python
def compute_centroidDistances(x, centroids):
    return np.sum((centroids - x)**2, axis = 1)

# `data_rdd` is an RDD
# `centroids` is a numpy array
clusterDistances_rdd = data_rdd \
    .map(lambda x: compute_clusterDistances(x, centroids))
```

### Load and preprocess the dataset

In [ ]:
# fetch the dataset and its labels
kdd_data, kdd_labels = fetch_kddcup99(
    percent10 = True,
    shuffle = True,
    return_X_y = True
)
# transform bytes entries into integers
entries_dict = {
    i: np.unique(kdd_data[:,i], return_inverse=True) 
        for i in range(kdd_data.shape[1]) 
            if isinstance(kdd_data[0,i], bytes) 
}
for key, values in entries_dict.items():
    kdd_data[:,key] = values[1]
# and then cast everything into a float
kdd_data = kdd_data.astype(float)

In [7]:
#Real dataset

# Change percent10 to 'False' to fetch the full dataset (4M rows)
# for local works is better to leave it as 'True'
kdd = fetch_kddcup99(shuffle=True,percent10=True) 
kdd_data = kdd.data

# Remove string features and standardize them
data = np.delete(kdd_data,np.arange(1,4,1),axis = 1) 
scaler = StandardScaler()
data = scaler.fit_transform(data)

#parallelize
data_rdd = sc.parallelize([row for row in data])
data_rdd = data_rdd.persist()

In [ ]:
np.unique_counts(kdd.target)

UniqueCountsResult(values=array([b'back.', b'buffer_overflow.', b'ftp_write.', b'guess_passwd.',
       b'imap.', b'ipsweep.', b'land.', b'loadmodule.', b'multihop.',
       b'neptune.', b'nmap.', b'normal.', b'perl.', b'phf.', b'pod.',
       b'portsweep.', b'rootkit.', b'satan.', b'smurf.', b'spy.',
       b'teardrop.', b'warezclient.', b'warezmaster.'], dtype=object), counts=array([  2203,     30,      8,     53,     12,   1247,     21,      9,
            7, 107201,    231,  97278,      3,      4,    264,   1040,
           10,   1589, 280790,      2,    979,   1020,     20]))

In [62]:
np.unique_counts(kdd_data[:,2])

UniqueCountsResult(values=array([b'IRC', b'X11', b'Z39_50', b'auth', b'bgp', b'courier',
       b'csnet_ns', b'ctf', b'daytime', b'discard', b'domain',
       b'domain_u', b'echo', b'eco_i', b'ecr_i', b'efs', b'exec',
       b'finger', b'ftp', b'ftp_data', b'gopher', b'hostnames', b'http',
       b'http_443', b'imap4', b'iso_tsap', b'klogin', b'kshell', b'ldap',
       b'link', b'login', b'mtp', b'name', b'netbios_dgm', b'netbios_ns',
       b'netbios_ssn', b'netstat', b'nnsp', b'nntp', b'ntp_u', b'other',
       b'pm_dump', b'pop_2', b'pop_3', b'printer', b'private', b'red_i',
       b'remote_job', b'rje', b'shell', b'smtp', b'sql_net', b'ssh',
       b'sunrpc', b'supdup', b'systat', b'telnet', b'tftp_u', b'tim_i',
       b'time', b'urh_i', b'urp_i', b'uucp', b'uucp_path', b'vmnet',
       b'whois'], dtype=object), counts=array([    43,     11,     92,    328,    106,    108,    126,     97,
          103,    116,    116,   5863,    112,   1642, 281400,    103,
           99,    670, 

In [58]:
kdd_data[1,:]

array([0, b'icmp', b'ecr_i', b'SF', 1032, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 511, 511, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 255,
       255, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=object)

In [50]:
np.unique(data, axis = 0).shape

(141480, 38)

In [51]:
data.shape

(494021, 38)

In [32]:
data = np.random.rand(100, 2)
data_rdd = sc.parallelize([row for row in data])
data_rdd = data_rdd.persist()

In [8]:
def kMeansRandom_init(
    data_rdd: RDD,
    k: int
) -> npt.NDArray:
    centroids = np.array(
        data_rdd.takeSample(withReplacement=False, num=k)
    )
    return centroids

In [9]:
def kMeansPlusPlus_init(
    data: npt.NDArray,
    k: int,
    weights: npt.NDArray = np.array([])
) -> npt.NDArray:
    """
    Standard kMeans++ initialization method:
    given `data` (eventually weighted), returns `k` cluster centroids
    """
    if weights.shape[0] == 0:
        weights = np.ones(shape=(data.shape[0],1))
    
    centroids = data[np.random.randint(0, data.shape[0]),:].reshape(1, -1) # reshaping for easier stacking
    
    while (centroids.shape[0] < k):
        # since the original functions are made for map
        # we need to loop over the data
        minDistance_array = np.array(
            [get_minDistance(compute_centroidDistances(datum, centroids)) for datum in data]
        ) * weights # multiplyling by the weight simulates multiple copies of the same datum
        total_minDistance = np.sum(minDistance_array)
        # sampling probability proportional to minDistance
        new_centroid_idx = np.random.choice(minDistance_array.shape[0], size = 1, p = minDistance_array / total_minDistance)
        new_centroid = data[new_centroid_idx,:].reshape(1, -1)

        # edge case in which the same centroid is selected twice:
        # redo the iteration without saving the centroid
        if any(np.array_equal(new_centroid, row) for row in centroids): continue
        centroids = np.concatenate((centroids, new_centroid), axis = 0)

    return centroids

In [10]:
def kMeansNaive(
    data: npt.NDArray,
    centroids: npt.NDArray,
    epochs: int = 5
) -> npt.NDArray:
    """
    Standard kMeans algorithm:
    given `data`, updates the (k) `centroids` for `epochs` times,
    improving the clustering each time
    """
    k = centroids.shape[0]
    for _ in range(epochs):
        assignments = np.array(
            [get_clusterId(compute_centroidDistances(x, centroids)) for x in data]
        )
        centroids = np.array(
            [np.mean(data[assignments==i,:], axis = 0) for i in range(k)]
        )
    return centroids

In [11]:
def kMeansParallel_init(
    data_rdd: RDD,
    k: int,
    l: float
) -> npt.NDArray:
    """
    kMeans|| initialization method:
    returns `k` good `centroids`.
    `l` controls the probability of each point
    in `data_rdd` of being sampled as a pre-processed centroid.
    """

    centroids = np.array(
        data_rdd.takeSample(num=1, withReplacement=False)
    )
    
    minDistance_rdd = data_rdd \
        .map(lambda x: (x, get_minDistance(compute_centroidDistances(x, centroids)))) \
        .persist()

    cost = minDistance_rdd \
        .map(lambda x: x[1]) \
        .sum()

    iterations = int(np.ceil(np.log(cost))) if (cost > 1) else 1
    for _ in range(iterations):
        new_centroids = np.array(
            minDistance_rdd \
                .filter(lambda x: np.random.rand() < np.min((l * x[1] / cost, 1))) \
                .map(lambda x: x[0]) \
                .collect()
        )
        # edge case in which no new centroid is sampled:
        # this avoids the following `np.concatenate` to fail
        if len(new_centroids.shape) < 2:
            continue

        minDistance_rdd.unpersist()
        centroids = np.unique(
            np.concatenate((centroids, new_centroids), axis = 0), 
            axis = 0
        )

        minDistance_rdd = data_rdd \
            .map(lambda x: (x, get_minDistance(compute_centroidDistances(x, centroids)))) \
            .persist()
        cost = minDistance_rdd \
            .map(lambda x: x[1]) \
            .sum()
    
    minDistance_rdd.unpersist()
    clusterCounts = data_rdd \
        .map(lambda x: (get_clusterId(compute_centroidDistances(x, centroids)), 1)) \
        .countByKey()
    
    clusterCounts = np.array([w[1] for w in clusterCounts.items()])
    centroids = kMeansNaive(
        centroids, 
        kMeansPlusPlus_init(centroids, k, clusterCounts)
    )
    
    return centroids

In [ ]:
k = 15
l = k * 10
centroids = kMeansParallel_init(data_rdd, k, l)

25/09/09 08:28:44 WARN TaskSetManager: Stage 24 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.
25/09/09 08:28:45 WARN TaskSetManager: Stage 25 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.


In [46]:
np.unique(centroids, axis = 0).shape

(10, 38)

In [22]:
def miniBatchKMeans(
    data_rdd: RDD,
    centroids: npt.NDArray,
    iterations: int = 10,
    batch_fraction: float = 0.1
) -> npt.NDArray:
    k = centroids.shape[0]
    clusterCounters = np.zeros((k,)) # 1 / learning_rate
    for iter in range(iterations):
        miniBatch_rdd = data_rdd \
            .sample(withReplacement=False, fraction=batch_fraction)
        miniBatch_rdd = miniBatch_rdd \
            .map(lambda x: (get_clusterId(compute_centroidDistances(x, centroids)), 1, x)) \
            .persist()
        
        # counting how many assigments per cluster
        clusterCounts_dict = miniBatch_rdd \
            .map(lambda x: (x[0], x[1])) \
            .countByKey()
        clusterCounts = np.array(
            [clusterCounts_dict[i] if i in clusterCounts_dict.keys() else 0 for i in range(k)]
        )
        clusterCounters += clusterCounts
        
        # edge case in which a cluster has no assignments:
        # if also its counter is zero the whole iteration is repeated
        if any(np.isclose(v, 0) for v in clusterCounters): 
            iter -= 1
            miniBatch_rdd.unpersist()
            continue
        # otherwise its count will be set to 1 to avoid division by 0 in the update step
        clusterCounts = np.where(clusterCounts >= 1, clusterCounts, 1)

        # summing all points assigned to the same cluster
        # (in the update step this will be divided by the counts 
        # in order to get the mean for every cluster).
        # A dict is used for convenience and consistency with clusterCounts
        clusterSums_dict = dict(miniBatch_rdd \
            .map(lambda x: (x[0], x[2])) \
            .reduceByKey(lambda x, y: x + y) \
            .collect()
        )
        # edge case in which a cluster has no assignments:
        # the centroid is returned instead of 0 
        # (which would have been the sum of its assigned points) 
        # in order to not update its position 
        # (note how the terms cancel out in the update step)
        clusterSums = np.array(
            [clusterSums_dict[i] if i in clusterSums_dict.keys() else centroids[i,:] for i in range(k)]
        )

        # update step: c <- (1 - eta) * c + eta * x_mean
        # (note x_mean = x_sums / c_count)
        centroids = (1 - 1 / clusterCounters).reshape(-1, 1) * centroids + \
                    (1 / (clusterCounters * clusterCounts)).reshape(-1, 1) * clusterSums
        
        miniBatch_rdd.unpersist()
        
    return centroids

In [23]:
final_centroids = miniBatchKMeans(data_rdd, centroids, 10, 0.1)

25/09/08 08:28:23 WARN TaskSetManager: Stage 40 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.
25/09/08 08:28:24 WARN TaskSetManager: Stage 41 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.
25/09/08 08:28:24 WARN TaskSetManager: Stage 42 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.
25/09/08 08:28:25 WARN TaskSetManager: Stage 43 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.
25/09/08 08:28:26 WARN TaskSetManager: Stage 44 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.
25/09/08 08:28:27 WARN TaskSetManager: Stage 45 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.
25/09/08 08:28:28 WARN TaskSetManager: Stage 46 contains a task of very large size (10124 KiB). The maximum recommended task size is 1000 KiB.

In [11]:
sc.stop()
spark.stop()

In [ ]:
# stopping the cluster
! $SPARK_HOME/sbin/stop-all.sh

worker2: stopping org.apache.spark.deploy.worker.Worker
worker3: stopping org.apache.spark.deploy.worker.Worker
worker1: stopping org.apache.spark.deploy.worker.Worker


25/09/09 20:16:13 ERROR TaskSchedulerImpl: Lost executor 2 on 10.67.22.202: Command exited with code 143
25/09/09 20:16:13 ERROR TaskSchedulerImpl: Lost executor 0 on 10.67.22.208: Worker shutting down
25/09/09 20:16:13 ERROR TaskSchedulerImpl: Lost executor 3 on 10.67.22.170: Command exited with code 143


master: stopping org.apache.spark.deploy.worker.Worker
stopping org.apache.spark.deploy.master.Master


25/09/09 20:16:14 WARN StandaloneAppClient$ClientEndpoint: Connection to 10.67.22.224:7077 failed; waiting for master to reconnect...
25/09/09 20:16:14 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
